In [161]:
import random
import copy

# Draughts Endgame : 4 Kings x 2 Kings

In the end I decided to create a game of drafts following an endgame based on the following:

https://lidraughts.org/study/YiAZbWM6

I decided to go with 4 kings vs 2 kings.

In this scenario, there will be only one 'type' of move. Kings are able to move diagonally forward and backwards, as well as capture other pieces.

## Draughts Class
Class which determines start position of pieces, board state, and display the board

In [ ]:
class CurrentBoard:

    BOARD_SIZE = 6
    PLAYER_COUNT = 6

    def __init__(self, board=None):
        if board is None:
            self.board = [[" " for _ in range(self.BOARD_SIZE)] for _ in range(self.BOARD_SIZE)]
        else:
            self.board = board
        #self.player_colour = player_colour
        self.player_positions = {"W": [], "B": []} # store positions of W & B players for use in all_possible_moves function
        self.state = self.state_of_board()
        #self.initial_board()

    # set up the initial board with the 6 pieces in the correct positions
    def initial_board(self):
        # randomise based on board size (64) and number of players (6)
        start_positons = random.sample(range(self.BOARD_SIZE*self.BOARD_SIZE), self.PLAYER_COUNT)

        # give the white players first two positions and black players last 4
        white_kings = start_positons[:2]
        black_kings = start_positons[2:]

        # assign the players their positions on the board by calculating the row and col
        for index in white_kings:
            row = index // self.BOARD_SIZE
            col = index % self.BOARD_SIZE
            self.board[row][col] = "W"
            self.player_positions["W"].append((row,col))
        
        # for index in black_kings:
        #     row = index // self.BOARD_SIZE
        #     col = index % self.BOARD_SIZE
        #     self.board[row][col] = "B"
        #     self.player_positions["B"].append((row,col))


    def display_board(self, game_display = False):
        print("+" + "---+" * self.BOARD_SIZE)
        for row in range(self.BOARD_SIZE):
            print("|", end="")
            for col in range(self.BOARD_SIZE):
                print(f" {self.board[row][col]} |", end="")
            print()
            print("+" + "---+" * self.BOARD_SIZE)

    def state_of_board(self):
        # to check the state of the board for a win, loss, or draw
        state = "Unfinished function"
    
        return state

    # In this case the only moves are king moves. They can move as outlined above
    def all_possible_moves(self, player_piece):
        possible_moves = []

        player_positions = self.player_positions[player_piece]
        print("All player positions", player_positions)

        move_directions = [(1, -1), (1, 1), (-1, -1), (-1, 1)]  # f-l, f-r, b-l, b-r

        for row, col in player_positions:
            print("Player position: ", row, col)  # debug

            for rowd, cold in move_directions:
                new_row, new_col = row + rowd, col + cold
                print("Making move to : ", new_row, new_col)  # debug

                for _ in range(self.BOARD_SIZE - 1):  # needs to check to max number of diagonal moves
                    print("current loop position: ", _)  # debug

                    if 0 <= new_row < self.BOARD_SIZE and 0 <= new_col < self.BOARD_SIZE:  # make sure within bounds of board
                        if self.board[new_row][new_col] == " ":
                            new_board = copy.deepcopy(self.board)
                            print("deep copy of board is :")
                            CurrentBoard(new_board).display_board()
                            new_player_positions = copy.deepcopy(self.player_positions)
                            new_board[row][col] = " "  # remove player from old position
                            new_board[new_row][new_col] = player_piece  # put the player in the new position
                            new_player_positions[player_piece] = [(new_row, new_col)]  # update player positions
                            possible_moves.append(CurrentBoard(new_board))
                        else:
                            break
                        new_row += rowd
                        new_col += cold
                    else:
                        break
                else:
                    continue  # Continue to the next direction
                break  # Break out of the directions loop if a move is found

        return possible_moves







In [322]:
cb = CurrentBoard()

In [323]:
cb.initial_board()

In [324]:
# check player positions
cb.player_positions

{'W': [(1, 2), (4, 4)], 'B': []}

In [325]:
# Show initial board
cb.display_board()

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   | W |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   | W |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


In [326]:

# check all moves fro player based on position
cb.all_possible_moves("W")[0].display_board()

All player positions [(1, 2), (4, 4)]
Player position:  1 2
Making move to :  2 1
current loop position:  0
deep copy of boaerd is :
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   | W |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   | W |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
current loop position:  1
deep copy of boaerd is :
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   | W |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   | W |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
current loop position:  2
deep copy of boaerd is :
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   | W

In [243]:
c

player space 0 3
available space 1 2
available space 1 4
available space -1 2
available space -1 4
player space 7 3
available space 8 2
available space 8 4
available space 6 2
available space 6 4
+---+---+---+---+---+---+---+---+
|   |   |   | W |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | W |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | W |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   | W |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+


In [198]:
"""  taken from searchtreeconnect4 :
https://github.com/Robert-Sheehy/AI/blob/main/2024/Search%20Tree/searchtreeconnect4_24.py
"""
class SearchTreeNode:

    def __init__(self,board_instance,playing_as, ply=0):
        self.children = []
        self.max_ply_depth = 3
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        if self.current_board.state == "U":
            if self.ply_depth<= self.max_ply_depth:
                self.generate_children()
            else:
                # evaluation function code
                pass

        else:   # Game over
            if self.current_board.state == "D":
                self.value = 0
            else:
                if ((self.ply_depth % 2) == 0):
                    self.value = -1000
                else:
                    self.value = 1000
            self.value_is_assigned = True

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children  = sorted(self.children, key = lambda x:x.min_max_value())

        if ((self.ply_depth % 2) == 0):
            # computers move
            self.value = self.children[-1].value
        else:
            #players move
            self.value = self.children[0].value
        self.value_is_assigned = True

        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(board_for_next_move, self.current_board.other(self.move_for), ply=self.ply_depth + 1))


In [196]:
stn = SearchTreeNode(cb, "W")

In [197]:
stn.children

[]

## Play the Game

In [17]:
def play_draughts():

    MAX_MOVES = 10

    response = input("Do you want to play first? (y/n)")
    players_turn = (response == "y")

    response = input("Do you want to play as black or white? (b/w)")
    cb = CurrentBoard()

    player_is_playing = cb.other(cb.other(response))

    for _ in range(MAX_MOVES):

        if players_turn:
            cb.display(game_display=True)

            while True:
                try:
                    choice = int(input("Make your move (Enter index of board): "))
                    if choice < 0 or choice >= 64:
                        print("Invalid index, please try again (0 and 63)")
                    elif cb.board[choice] != ' ':
                        print("Invalid index, the selected position is occupied")
                    else:
                        break
                except ValueError:
                    print("Invalid input, please enter a number")

            cb.move_piece(choice, player_is_playing)
            cb.display()
            
        else:
            search_tree = SearchTreeNode(cb, cb.other(player_is_playing))
            search_tree.min_max_value()
            cb = search_tree.children[-1].current_board
            cb.display()

        if cb.state != "U":
            if cb.state == "D":
                print("It's a draw")
            else:
                if players_turn:
                    print("You win")
                else:
                    print("You lose!")

                cb.display()
            break

        players_turn = not players_turn
